In [27]:
from nba_api.stats.endpoints import PlayerGameLog
import pandas as pd

def get_player_game_logs(player_id, season='All'):
    """Fetch game logs for a given player and season ('All' for career)."""
    if season == 'All':
        # Fetch all available game logs for the player
        game_logs = pd.DataFrame()
        for year in range(2003, 2025):  # Example range for LeBron (adjust as needed)
            season_logs = PlayerGameLog(player_id=player_id, season=str(year)).get_data_frames()[0]
            game_logs = pd.concat([game_logs, season_logs], ignore_index=True)
    else:
        # Fetch game logs for a specific season
        game_logs = PlayerGameLog(player_id=player_id, season=season).get_data_frames()[0]
    
    return game_logs

def calculate_rolling_and_historical_averages(game_logs):
    # Ensure game logs are sorted by GAME_DATE
    game_logs['GAME_DATE'] = pd.to_datetime(game_logs['GAME_DATE'])
    game_logs = game_logs.sort_values('GAME_DATE').reset_index(drop=True)
    
    # Define columns to calculate rolling averages for
    rolling_columns = ['PTS', 'MIN', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 
                       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PLUS_MINUS']
    
    # Calculate rolling averages for the last 5 games
    for col in rolling_columns:
        game_logs[f'Rolling_{col}'] = game_logs[col].rolling(window=5, min_periods=1).mean()
    
    # Identify home and away games
    game_logs['Home/Away'] = game_logs['MATCHUP'].apply(lambda x: 'Home' if 'vs' in x else 'Away')
    
    # Extract Opponent
    game_logs['OPP'] = game_logs['MATCHUP'].apply(lambda x: x.split()[-1])
    
    return game_logs, rolling_columns

def calculate_career_averages(game_logs):
    """Calculate career averages against each opponent."""
    career_columns = ['PTS', 'MIN', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 
                      'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PLUS_MINUS']
    
    # Calculate career averages against each opponent
    historical_averages = game_logs.groupby('OPP')[career_columns].mean().reset_index()
    
    return historical_averages

# Example usage
player_id = 2544  # Example: LeBron James' ID
season = '2024'

# Fetch career data and current season data from NBA API
game_logs_career = get_player_game_logs(player_id=player_id, season='All')
game_logs_season = get_player_game_logs(player_id=player_id, season=season)

# Calculate rolling averages for the current season
rolling_averages, rolling_columns = calculate_rolling_and_historical_averages(game_logs_season)

# Calculate career averages against each opponent
career_averages = calculate_career_averages(game_logs_career)

print("Rolling Averages (Last 5 Games):")
print(rolling_averages[['GAME_DATE', 'MATCHUP', 'Home/Away'] + [f'Rolling_{col}' for col in rolling_columns]])

print("\nCareer Averages vs Each Opponent:")
print(career_averages)


/var/folders/8p/x8n81b411l5gvyf3207yrd1r0000gn/T/ipykernel_66735/3801143201.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  game_logs['GAME_DATE'] = pd.to_datetime(game_logs['GAME_DATE'])


KeyError: 'OPP'